<a href="https://colab.research.google.com/github/thamizhannal/eva5/blob/master/EVA5_Session_2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import pytoruch packages
from __future__ import print_function
# import pytorch main package
import torch
# import Base class for all neural network modules.
import torch.nn as nn
import torch.nn.functional as F
# import torch.optim is a package implementing various optimization algorithms.
import torch.optim as optim
# import torchvision packge & dataset.
# The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
# All datasets are subclasses of torch.utils.data.Dataset 
from torchvision import datasets, transforms


In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Help Conv2d syntex with default values
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # Before convolution RF:1x1
        #input: 28x28x1  OUtput: 28x28x1 RF:3x3 (3x3 increase RF by 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1) 
        
        #input: 28x28x32  OUtput: 28x28x64 RF:5x5 (3x3 increase RF by 2)
        # No of parameter: 3*3*32(in_channels)*64(out_channels) = 18432 + bias(64) = 18496
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) 
        
        #input: 14x14x64  OUtput: 14x14x64 RF:10x10 (2x2 MP multiply RF by 2 or double the existing RF)
        self.pool1 = nn.MaxPool2d(2, 2)  

        #input: 14x14x64  OUtput: 14x14x128 RF:12x12 (3x3 increase RF by 2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) 
        
        #input: 14x14x128  OUtput: 14x14x256 RF:14x14 (3x3 increase RF by 2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) 
        
        #input: 14x14x256  OUtput:7x7x256 RF:28x28 (2x2 MP multiply RF by 2 or double the existing RF)
        self.pool2 = nn.MaxPool2d(2, 2) 
        
        #input:7x7x256  OUtput: 5x5x512 RF:30x30  (3x3 increase RF by 2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3) 
        
        #input:5x5x512  OUtput:3x3x1024 RF:32x32 (3x3 increase RF by 2)
        self.conv6 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3) 
        
        #input:3x3x1024   OUtput: 1x1x10 
        self.conv7 = nn.Conv2d(in_channels=1024, out_channels=10, kernel_size=3) 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        print(x.view(-1, 10))
        # view perform reshape operation. Here convert x as 1x10 matrics
        x = x.view(-1, 10)       
        # Applies the \log(\text{Softmax}(x))log(Softmax(x)) function to an n-dimensional input Tensor
        return F.log_softmax(x)

In [14]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

tensor([[0.0000, 0.0000, 0.0020, 0.0061, 0.0087, 0.0020, 0.0009, 0.0000, 0.0000,
         0.0015],
        [0.0000, 0.0000, 0.0027, 0.0052, 0.0091, 0.0023, 0.0016, 0.0000, 0.0000,
         0.0017]], device='cuda:0', grad_fn=<ViewBackward>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [16]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Negative log likelihood
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=2.303062915802002 batch_id=1:   0%|          | 2/469 [00:00<01:19,  5.88it/s]

tensor([[0.0000, 0.0000, 0.0066,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0088,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0066,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0083,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0100,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0074,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000, 0.0000, 0.0075,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0090,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0054,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0053,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0034,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0060,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)


loss=2.3020341396331787 batch_id=3:   1%|          | 4/469 [00:00<01:04,  7.26it/s]

tensor([[0.0000e+00, 0.0000e+00, 6.0579e-05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.6204e-03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.1203e-03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 5.2686e-03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.3501e-03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.0766e-03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0025,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.002

loss=2.302138328552246 batch_id=5:   1%|          | 5/469 [00:00<00:59,  7.85it/s]

...

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)


loss=1.776332974433899 batch_id=468: 100%|██████████| 469/469 [00:38<00:00, 12.69it/s]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.6749,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.5281,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.6425,  0.000

loss=1.776332974433899 batch_id=468: 100%|██████████| 469/469 [00:38<00:00, 12.32it/s]


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0